# Resolvers

Values that compute themselves: lazy loading, API calls, computed values.

In [ ]:
# Setup: Install genro-bag (run this cell first on Colab)\n!pip install -q genro-bag

In [ ]:
from genro_bag import Bag
from genro_bag.resolvers import BagCbResolver, UrlResolver

## The Core Idea

Instead of storing a value directly, a node can store a **resolver** that computes the value on demand.

In [ ]:
from datetime import datetime

def get_timestamp():
    """Returns current timestamp - called each time the value is accessed."""
    return datetime.now().isoformat()

bag = Bag()
bag['timestamp'] = BagCbResolver(get_timestamp)

# Each access calls the function
print(f"First access:  {bag['timestamp']}")
import time; time.sleep(0.1)
print(f"Second access: {bag['timestamp']}")

## Caching

Control how often values are recomputed with `cache_time`.

In [ ]:
call_count = 0

def expensive_computation():
    global call_count
    call_count += 1
    return {'result': 42, 'calls': call_count}

bag = Bag()

# cache_time=0: compute every time (default)
# cache_time=60: cache for 60 seconds
# cache_time=-1: cache forever
bag['cached'] = BagCbResolver(expensive_computation, cache_time=60)

print(f"First:  {bag['cached']}")
print(f"Second: {bag['cached']}")
print(f"Third:  {bag['cached']}")
print(f"\nFunction was called {call_count} time(s)")

## URL Resolver

Fetch data from HTTP endpoints.

In [ ]:
bag = Bag()

# Fetch JSON from an API (cached for 5 minutes by default)
bag['users'] = UrlResolver(
    'https://jsonplaceholder.typicode.com/users',
    as_bag=True,  # Parse JSON response as a Bag
    cache_time=300
)

# Access triggers the HTTP request
users = bag['users']
print(f"Type: {type(users)}")
print(f"Number of users: {len(users)}")

# Navigate the result
first_user = users[0]
print(f"First user name: {first_user['name']}")

## Custom Resolvers

Create your own resolver by extending `BagResolver`.

In [ ]:
from genro_bag.resolver import BagResolver

class ConfigFileResolver(BagResolver):
    """Load configuration from a file."""
    
    class_args = ['filepath']  # Positional arguments
    class_kwargs = {'cache_time': -1}  # Cache forever by default
    
    def load(self):
        # In real code, you'd read and parse the file
        filepath = self._kw['filepath']
        return {'loaded_from': filepath, 'debug': True, 'level': 'INFO'}

bag = Bag()
bag['config'] = ConfigFileResolver('/etc/myapp/config.json')

print(f"Config: {bag['config']}")

## Static Access

Check if a value is cached without triggering the resolver.

In [ ]:
bag = Bag()
bag['lazy'] = BagCbResolver(lambda: 'computed!')

# Check without triggering
cached = bag.get_item('lazy', static=True)
print(f"Before access (static=True): {cached}")

# Now trigger it
value = bag['lazy']
print(f"After access: {value}")

# Now static access returns the cached value
cached = bag.get_item('lazy', static=True)
print(f"After access (static=True): {cached}")

## Serialization

Resolvers survive serialization with TYTX.

In [ ]:
bag = Bag()
bag['data'] = BagCbResolver(lambda: {'computed': True})

# Serialize
tytx = bag.to_tytx()

# Restore
restored = Bag.from_tytx(tytx)

# The resolver is preserved!
print(f"Restored value: {restored['data']}")

## Key Takeaways

- **Lazy loading**: Values computed on demand
- **Caching**: Control with `cache_time` (0=never, >0=seconds, -1=forever)
- **Built-in**: `BagCbResolver` (callbacks), `UrlResolver` (HTTP)
- **Custom**: Extend `BagResolver`, implement `load()`
- **Transparent**: Access resolvers like regular values